In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split



In [21]:
rating = pd.read_csv('rating.csv')
anime = pd.read_csv('anime.csv')
# drop rows with rating -1
rating = rating[rating.rating != -1]

genres = set()
for i in anime['genre']:
    if type(i) != float:
        for j in i.split(', '):
            genres.add(j)
genres = list(genres)
anime = anime.drop(['type', 'episodes', 'rating', 'members'], axis=1)
# add generes list as columns in anime dataframe
ad = pd.DataFrame(columns=genres)
anime = pd.concat([anime, ad], axis=1)
# add 1 if anime has that genre else 0
for i in range(len(anime)):
    if type(anime['genre'][i]) != float:
        for j in anime['genre'][i].split(', '):
            anime[j][i] = 1
# drop unnecessary columns
anime = anime.drop(['genre'], axis=1)
# fillna
anime = anime.fillna(0)
print(anime.head())

C:\Users\Dell Latitude E5470\AppData\Local\Temp\ipykernel_4812\4254610191.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anime[j][i] = 1


   anime_id                              name  Music  Magic  Game  Fantasy  \
0     32281                    Kimi no Na wa.      0      0     0        0   
1      5114  Fullmetal Alchemist: Brotherhood      0      1     0        1   
2     28977                          Gintama°      0      0     0        0   
3      9253                       Steins;Gate      0      0     0        0   
4      9969                     Gintama&#039;      0      0     0        0   

   Thriller  Vampire  Kids  Hentai  ...  Parody  Harem  Police  Ecchi  \
0         0        0     0       0  ...       0      0       0      0   
1         0        0     0       0  ...       0      0       0      0   
2         0        0     0       0  ...       1      0       0      0   
3         1        0     0       0  ...       0      0       0      0   
4         0        0     0       0  ...       1      0       0      0   

   Historical  Shoujo  School  Space  Horror  Adventure  
0           0       0       1     

In [30]:
# merge anime and rating
merged = pd.merge(rating, anime, on='anime_id')
merged = merged.drop(['name'], axis=1)
merged


,user_id,anime_id,rating,Music,Magic,Game,Fantasy,Thriller,Vampire,Kids,...,Parody,Harem,Police,Ecchi,Historical,Shoujo,School,Space,Horror,Adventure
0,1,8074,10,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,3,8074,6,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,5,8074,2,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,12,8074,6,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,14,8074,6,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6337234,69964,23585,7,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6337235,69964,33659,6,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6337236,72800,30738,4,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
6337237,73135,8723,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
# normalize rating
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
merged['rating'] = scaler.fit_transform(merged['rating'].values.reshape(-1,1))
merged.head()


,user_id,anime_id,rating,Music,Magic,Game,Fantasy,Thriller,Vampire,Kids,...,Parody,Harem,Police,Ecchi,Historical,Shoujo,School,Space,Horror,Adventure
0,1,8074,1.000000,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,3,8074,0.555556,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,5,8074,0.111111,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,12,8074,0.555556,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,14,8074,0.555556,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [46]:
# split data into train and test
train, test = train_test_split(merged, test_size=0.2, random_state=42)
print(train.shape)
print(test.shape)




MemoryError: Unable to allocate 1.70 GiB for an array with shape (45, 5069791) and data type int64